In [1]:
import pandas as pd
import numpy as np


# from google.colab import drive
# drive.mount('/content/drive')


In [2]:
df = pd.read_csv('../raw_data/raw_en.csv')

/tmp/ipykernel_3352/2459300368.py:1: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../raw_data/raw_en.csv')


In [3]:
df.shape

(457074, 8)

In [4]:
def convert_stars(row):
    if not pd.isnull(row['stars']):
        if row['stars'] > 3:
            return 'Recommended'
        return 'Not Recommended'
    return row['recommendation']
    

In [5]:
df['recommendation'] = df.apply(convert_stars, axis = 1)

In [6]:
df.tail(50)

,name,date,recommendation,review,playing_hours,source,stars,language
457024,zhexue-daolun,NaN,Recommended,Inspiring people to think,NaN,coursera,5.0,en
457025,zhichang-suyang,NaN,Recommended,"very useful, very valuable.I had learn so many...",NaN,coursera,5.0,en
457026,zhichang-suyang,NaN,Recommended,Great,NaN,coursera,5.0,en
457027,zhichang-suyang,NaN,Recommended,I love this course and learn a lot from it. St...,NaN,coursera,5.0,en
457028,zhichang-suyang,NaN,Recommended,"Great course regarding professionalism, it tou...",NaN,coursera,5.0,en
457029,zhichang-suyang,NaN,Recommended,Wonderful lesson which should be promoted in a...,NaN,coursera,5.0,en
457030,zhichang-suyang,NaN,Recommended,Very useful! I have learned a lot from this co...,NaN,coursera,5.0,en
457031,zhichang-suyang,NaN,Recommended,It worth spend your time to study and repeat a...,NaN,coursera,5.0,en
457032,zhichang-suyang,NaN,Recommended,Nice course focusing on the essentials in work...,NaN,coursera,4.0,en
457033,zhichang-suyang,NaN,Recommended,"Really enjoy this course,and like the professor.",NaN,coursera,5.0,en


In [29]:
df_not_recommended = df[df['recommendation'] == 'Not Recommended']
from gensim.models import Word2Vec
X_train = df_not_recommended.head(10000)['review']
X_test = df_not_recommended.tail(1000)['review']
word2vec = Word2Vec(sentences=X_train, vector_size = 100, min_count = 10, window = 6)

In [30]:
X_train

12       Amount of bugs and glitches from error during ...
19       i would not recommend this.. this game used to...
26       I couldn't even start a gamewith My friends. O...
27       The game is completely unbalanced it doesn't m...
33                                  this game is dog s hit
                               ...                        
50210    Constantly crashes on a high end terminal. Res...
50211                              How much money you got?
50212                                 Physics are crap lol
50217    Honestly when i started this game it was fun a...
50237                               can i refund this game
Name: review, Length: 10000, dtype: object

In [31]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)


# Pad the training and test embedded sentences
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)

In [32]:
X_train_pad[0][0]

array([-3.5668197e-01, -1.0035462e-01, -7.3657125e-02, -4.7607812e-01,
        4.3416411e-01,  9.6091464e-02,  2.1099460e-01, -1.0169571e-01,
       -6.3002992e-01,  2.5718528e-01, -5.5816054e-01,  3.0465093e-02,
       -5.3394186e-01, -2.9645509e-01,  5.7352994e-02,  1.9006287e-01,
        4.1985801e-01, -5.8668238e-01,  1.2117300e-01,  3.8626063e-01,
       -8.7962314e-02,  5.1249176e-02,  8.9103359e-01,  6.8837488e-01,
       -2.7915746e-01,  2.2496128e-01,  1.5336223e-01,  1.1644909e-01,
        2.7969062e-01,  2.0238650e-01,  4.8326176e-01, -1.3670628e-01,
       -7.6164287e-01,  1.4544466e-04, -3.4322169e-01,  4.8545852e-01,
        4.3954480e-02, -3.9781552e-02, -2.3314089e-02, -2.6607072e-01,
        3.9634485e-02,  2.0401867e-01,  2.1212146e-01,  9.6161207e-03,
       -2.1691270e-01,  4.2891812e-01, -1.5417603e-01, -2.8370976e-01,
       -4.3403739e-01,  4.6387957e-03, -2.2318797e-01, -5.4910207e-01,
       -2.8513557e-01, -2.2320358e-01,  1.8185526e-01, -5.1167774e-01,
      

In [33]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import  Dense, Masking, LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
def build_encoder(latent_dimension):
    '''returns an encoder model, of output_shape equals to latent_dimension'''
    encoder = Sequential()
    
    encoder.add(Masking(mask_value=0))
    encoder.add(LSTM(latent_dimension, activation="tanh"))
    
    return encoder

In [34]:
from tensorflow.keras.layers import Input
encoder = build_encoder(2)
#encoder.build((None,200,100))
#encoder.summary()

In [35]:
def build_decoder(latent_dimension):
    # $CHALLENGIFY_BEGIN
    decoder = Sequential()
    decoder.add(RepeatVector(200)) #RepeatVector add comments
    decoder.add(LSTM(100, activation='tanh', return_sequences=True))
    decoder.add(TimeDistributed(Dense(100))) #TimeDistrivuted add comments

    return decoder


In [36]:
decoder = build_decoder(2)
# decoder.build((None,200,100))
# decoder.summary()

In [37]:
X_train_pad.shape[1:]

(200, 100)

In [38]:
from tensorflow.keras import Model


def build_autoencoder(encoder, decoder):
    inp = Input(X_train_pad.shape[1:])
    encoded = encoder(inp)
    decoded = decoder(encoded)
    autoencoder = Model(inp, decoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder

In [39]:
autoencoder = build_autoencoder(encoder, decoder)
#autoencoder.summary()

In [41]:
autoencoder.fit(X_train_pad, X_train_pad, epochs=100, verbose=1)

Epoch 1/100
313/313 [==============================] - 153s 487ms/step - loss: 0.1002
Epoch 2/100
313/313 [==============================] - 135s 432ms/step - loss: 0.1002
Epoch 3/100
313/313 [==============================] - 143s 457ms/step - loss: 0.1000
Epoch 4/100
313/313 [==============================] - 142s 454ms/step - loss: 0.0997
Epoch 5/100
313/313 [==============================] - 149s 476ms/step - loss: 0.0982
Epoch 6/100
313/313 [==============================] - 135s 431ms/step - loss: 0.0965
Epoch 7/100
313/313 [==============================] - 135s 431ms/step - loss: 0.0957
Epoch 8/100
313/313 [==============================] - 138s 439ms/step - loss: 0.0953
Epoch 9/100
313/313 [==============================] - 134s 428ms/step - loss: 0.0949
Epoch 10/100
313/313 [==============================] - 134s 427ms/step - loss: 0.0946
Epoch 11/100
313/313 [==============================] - 137s 438ms/step - loss: 0.0944
Epoch 12/100
313/313 [==============================

313/313 [==============================] - 197s 631ms/step - loss: 0.0919
Epoch 96/100
313/313 [==============================] - 248s 794ms/step - loss: 0.0916
Epoch 97/100
313/313 [==============================] - 183s 585ms/step - loss: 0.0916
Epoch 98/100
313/313 [==============================] - 167s 534ms/step - loss: 0.0915
Epoch 99/100
313/313 [==============================] - 176s 561ms/step - loss: 0.0914
Epoch 100/100
313/313 [==============================] - 168s 536ms/step - loss: 0.0915


In [ ]:
from tensorflow.keras import layers, Sequential


model = Sequential()
model.add(layers.Masking(mask_value=0))
model.add(layers.LSTM(20, activation="tanh"))
model.add(layers.Dense(2, activation="relu"))
model.add(LSTM(20, activation='tanh', return_sequences=True))

In [ ]:
# lstm autoencoder recreate sequence
from numpy import array
from tensorflow import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import LSTM
from keras.layers import Dense


from keras.utils.vis_utils import plot_model


# define input sequence
sequence = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
# reshape input into [samples, timesteps, features]
n_in = len(sequence)
sequence = sequence.reshape((1, n_in, 1))
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_in,1)))
model.add(RepeatVector(n_in))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(sequence, sequence, epochs=300, verbose=0)
#connect the encoder LSTM as the output layer
model = Model(inputs=model.inputs, outputs=model.layers[0].output)
#plot_model(model, show_shapes=True, to_file='lstm_encoder.png')
# get the feature vector for the input sequence
#yhat = model.predict(sequence)
#print(yhat.shape)
#print(yhat)

In [ ]:
model.summary()

In [ ]:
!pip install pydot